In [1]:
#
# Xavier Initialized MLP MNIST
#
# @author becxer
# @email becxer87@gmail.com
# @reference https://github.com/sjchoi86/Tensorflow-101
#

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  

print ("packages are loaded")

packages are loaded


In [2]:
# Get MNIST data set from tensorflow
from tensorflow.examples.tutorials.mnist import input_data
mnist_set = input_data.read_data_sets('images/MNIST/', one_hot = True)

train_img = mnist_set.train.images
train_label = mnist_set.train.labels
test_img = mnist_set.test.images
test_label = mnist_set.test.labels

Extracting images/MNIST/train-images-idx3-ubyte.gz
Extracting images/MNIST/train-labels-idx1-ubyte.gz
Extracting images/MNIST/t10k-images-idx3-ubyte.gz
Extracting images/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
# Options for training
learning_rate = 0.001
training_epochs = 50
batch_size = 100
display_step = 10

# Options for Hidden Network Nodes
n_hidden = [256, 256, 256, 256] # hidden layer's node count
w_dev = 0.1
drop_out_ratio = 0.7

In [4]:
# Xavier Initializer
def xavier_initializer(input_size, output_size, uniform=True):
    if uniform:
        mx = tf.sqrt(6.0/(input_size + output_size))
        return tf.random_uniform_initializer(-mx,mx)
    else:
        mx = tf.sqrt(3.0/(input_size + output_size))
        return tf.truncated_normal_initializer(stddev=mx)

In [5]:
# Build Graph of Xavier Initialized MLP

# Define placeholder & Variables
x = tf.placeholder("float", [None, train_img.shape[1]])
y = tf.placeholder("float", [None, train_label.shape[1]])
drop_out_prob = tf.placeholder("float")

def one_layer(_x, _W, _b, _dop):
    return tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(_x, _W),_b)),_dop)

WS = {}
BS = {}

last_input_layer = x
last_input_layer_size = train_img.shape[1]
for idx, hl_size in enumerate(n_hidden):
    _W = tf.get_variable(\
                         'h_'+str(idx), shape=[last_input_layer_size, hl_size],\
                         initializer=xavier_initializer(last_input_layer_size, hl_size))
    _b = tf.Variable(tf.random_normal([hl_size])) 
    
    last_input_layer = one_layer(last_input_layer, _W, _b, drop_out_prob)
    last_input_layer_size = hl_size
    
    WS['h_' + str(idx)] = _W
    BS['b_' + str(idx)] = _b

    
WS['out'] = tf.get_variable(\
                            'out', shape=[last_input_layer_size, train_label.shape[1]],\
                            initializer=xavier_initializer(last_input_layer_size, train_label.shape[1]))
BS['out'] = tf.Variable(tf.random_normal([train_label.shape[1]], stddev=w_dev))

# Define operators

out = one_layer(last_input_layer, WS['out'], BS['out'], 1.0)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

pred = tf.argmax(out, 1)
accr = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(y, 1)),"float"))

init = tf.initialize_all_variables()
print ("Graph build")

Graph build


In [6]:
# Training Graph
sess = tf.Session()
sess.run(init)

for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(train_img.shape[0]/batch_size)
    for i in range(num_batch):
        randidx = np.random.randint(train_img.shape[0], size=batch_size)
        batch_xs = train_img[randidx, :]
        batch_ys = train_label[randidx, :]                
            
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, drop_out_prob : drop_out_ratio})
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, drop_out_prob : 1.})/num_batch
            
    if epoch % display_step == 0:
        train_img_acc = sess.run(accr , ({x: batch_xs, y: batch_ys, drop_out_prob : 1.}))
        print ("epoch: %03d/%03d  ,  cost: %.6f  ,  train_img_acc: %.3f" \
               % (epoch, training_epochs, avg_cost, train_img_acc))

print ("Training complete, Accuracy : %.6f" \
        % (sess.run(accr, {x : test_img, y : test_label, drop_out_prob : 1.}),))

epoch: 000/050  ,  cost: 1.904273  ,  train_img_acc: 0.700
epoch: 010/050  ,  cost: 0.029600  ,  train_img_acc: 0.990
epoch: 020/050  ,  cost: 0.012155  ,  train_img_acc: 1.000
epoch: 030/050  ,  cost: 0.007493  ,  train_img_acc: 1.000
epoch: 040/050  ,  cost: 0.004715  ,  train_img_acc: 1.000
Training complete, Accuracy : 0.981600
